## 축소전 데이터와 PCA 이후 데이터 Training 시간 비교

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')


In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.shape

(42000, 785)

In [4]:
label = data.iloc[:,0]
X = data.iloc[:,1:]
#X = StandardScaler().fit_transform(X)

In [5]:
data.shape

(42000, 785)

In [ ]:
X_tr, X_test, y_tr, y_test = train_test_split(X, label, test_size = 0.2, random_state = 4)

In [ ]:
X_tr.shape, y_tr.shape

In [ ]:
X_test.shape, y_test.shape

### 1) 축소 전 데이터 KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
# Train 
time_start = time.time()
knn.fit(X_tr,y_tr)
print('knn done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
time_start = time.time()
y_pred = knn.predict(X_test)
print('knn done! Time elapsed: {} seconds'.format(time.time()-time_start))
# test accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))


### 2) 축소 전 데이터 SVM

In [ ]:
#Create a svm Classifier
svm = SVC(kernel='rbf',C=3)

time_start = time.time()
#Train
svm.fit(X_tr, y_tr)
print('svm done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
#Predict the response for test dataset

time_start = time.time()
y_pred = svm.predict(X_test)
print('svm done! Time elapsed: {} seconds'.format(time.time()-time_start))
# test accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))


### PCA 로 차원축소

In [ ]:
pca = PCA(2)
pca_features = pca.fit_transform(X)
pca_features.shape

### 1) 축소 후 데이터 KNN

In [ ]:
X_tr, X_test, y_tr, y_test = train_test_split(X, label, test_size = 0.2, random_state = 34)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
# Train 
time_start = time.time()
knn.fit(X_tr,y_tr)
print('knn done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
time_start = time.time()
y_pred = knn.predict(X_test)
print('knn done! Time elapsed: {} seconds'.format(time.time()-time_start))
# test accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))


### 2) 축소 후 데이터 SVM

In [ ]:
#Create a svm Classifier
svm = SVC(kernel='rbf',C=3)

time_start = time.time()
#Train
svm.fit(X_tr, y_tr)
print('svm done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
#Predict the response for test dataset

time_start = time.time()
y_pred = svm.predict(X_test)
print('svm done! Time elapsed: {} seconds'.format(time.time()-time_start))
# test accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))
